## Data Load

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
train = pd.read_csv(r'/content/drive/My Drive/Projects/Home Credit Default Risk/train.csv')
test = pd.read_csv(r'/content/drive/My Drive/Projects/Home Credit Default Risk/test.csv')

In [0]:
train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,EXT_SOURCE_3,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,YEARS_EMPLOYED,YEARS_BIRTH,FAMILY SIZE,IS_CHILDREN,credit_income_ratio,annuity_income_ratio
0,1.0,Cash loans,1,0,Y,0.0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-3648.0,-2120.0,1.0,1.0,0.0,1.0,1.0,0.0,Laborers,1.0,2.0,2.0,WEDNESDAY,10.0,0.0,0.0,0.0,0.0,0.0,0.0,Business Entity Type 3,0.262949,0.139376,0.014900,No,2.0,2.0,2.0,2.0,-1134.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00000,1.745205,25.920548,Single,0,2.007889,0.121978
1,0.0,Cash loans,0,0,N,0.0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-1186.0,-291.0,1.0,1.0,0.0,1.0,1.0,0.0,Core staff,2.0,1.0,1.0,MONDAY,11.0,0.0,0.0,0.0,0.0,0.0,0.0,School,0.622246,0.535276,0.071400,No,1.0,0.0,1.0,0.0,-828.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,3.254795,45.931507,Small,0,4.790750,0.132217
2,0.0,Revolving loans,1,1,Y,0.0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-4260.0,-2531.0,1.0,1.0,1.0,1.0,1.0,0.0,Laborers,1.0,2.0,2.0,MONDAY,9.0,0.0,0.0,0.0,0.0,0.0,0.0,Government,0.555912,0.729567,0.095497,EMERGENCYSTATE_MODE,0.0,0.0,0.0,0.0,-815.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.616438,52.180822,Single,0,2.000000,0.100000
3,0.0,Cash loans,0,0,Y,0.0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-9833.0,-2437.0,1.0,1.0,0.0,1.0,0.0,0.0,Laborers,2.0,2.0,2.0,WEDNESDAY,17.0,0.0,0.0,0.0,0.0,0.0,0.0,Business Entity Type 3,0.650442,0.535276,0.090204,EMERGENCYSTATE_MODE,2.0,0.0,2.0,0.0,-617.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.05337,8.326027,52.068493,Small,0,2.316167,0.219900
4,0.0,Cash loans,1,0,Y,0.0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-4311.0,-3458.0,1.0,1.0,0.0,1.0,0.0,0.0,Core staff,1.0,2.0,2.0,THURSDAY,11.0,0.0,0.0,0.0,0.0,1.0,1.0,Religion,0.322738,0.535276,0.090284,EMERGENCYSTATE_MODE,0.0,0.0,0.0,0.0,-1106.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,8.323288,54.608219,Single,0,4.222222,0.179963


In [0]:
# label encoding snippet 
# label encoding will assign numerical values to each object value based on alphabetical order
# eg. A will be encoded as 0, B - 1, C- 2
# Helpful for High Cardinality Data - LightGBM can handle integer categorical fields

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [0]:
def label_encode(x):
  le.fit(train[x])
  train[x] = le.transform(train[x])
  test[x] = le.transform(test[x])

In [0]:
label_encode('FLAG_OWN_REALTY')

In [0]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [0]:
missing_cols = set( train.columns ) - set( test.columns )

for c in missing_cols:
    test[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
test = test[train.columns]

In [0]:
test.drop(columns = 'TARGET', inplace = True)

In [0]:
from sklearn.model_selection import train_test_split
xtrain, xval, ytrain, yval = train_test_split(train.iloc[:,1:], train['TARGET'], test_size = 0.2, random_state = 42)

## Random Forest

In [0]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators = 1000, random_state = 50, class_weight = 'balanced', n_jobs = -1) #n_jobs = -1 meaning utilize all system resources

In [0]:
# Fit the model
rf_model = random_forest.fit(train.iloc[:,1:], train['TARGET'])

In [0]:
# load submission file
submission = pd.read_csv(r'/content/drive/My Drive/Projects/Home Credit Default Risk/sample_submission.csv')

submission['TARGET'] = pd.DataFrame(random_forest.predict_proba(test))[1]
submission.to_csv('RandomForest_pred.csv',index = False)

In [0]:
# Feature importannce
import seaborn as sns; 
ft_imp = pd.Series(data=rf_model.feature_importances_, index=xtrain.columns).sort_values(ascending=False)[:15]
plt.figure(figsize=(5,5))
plt.title("Feature importance")
ax = sns.barplot(y=ft_imp.index, x=ft_imp.values, palette="Blues_d", orient='h')

## XGBoost

In [0]:
import xgboost as xgb

In [0]:
params = {
    'max_depth': 7, # to control over-fitting, optimal value range: 3-10
    'gamma' :0, # the minimum loss reduction required to make a split, XGB will build the tree first then prune in case of 0 gamma
    'eta':.03, # shrinkage factor, optimal value range: 0.01-0.2
    'subsample': 0.8, # fraction of observations to be randomly samples for each tree, optimal value range: 0.5-1
    'colsample_bytree': 0.8, # fraction of columns to be randomly samples for each tree, optimal value range: 0.5-1
    'scale_pos_weight' : 11.0, # for imbalanced data
    'objective':'binary:logistic', # loss function to be minimized
    'eval_metric':'auc', # metric to be used for validation data
}

In [0]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,
                    num_boost_round=500, # number of boosting rounds or trees to build
                    early_stopping_rounds=10, # stop if performance haven’t improved for N rounds
                    evals=[(matrix_test,'test')]
                   )
    return model

In [0]:
# train final model using tuned parameters
xgb_model = XGBmodel(xtrain,xval,ytrain,yval,params)

In [0]:
pred_xgb = xgb_model.predict(xgb.DMatrix(test), ntree_limit = xgb_model.best_ntree_limit).tolist()
submission['TARGET'] = pred_xgb
submission.to_csv('xgb_pred.csv',index = False)